# V16 (gated 82+83): Model competition

Re-runs V16 on the gated (82+83) dataset to check whether non-transit models still fit better after removing inconsistent sectors.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

sectors_used = [82, 83]
ds = sh.load_dataset()
lc = sh.stitch_pdcsap_sectors(ds, sectors_used)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V16')

print(json.dumps({
  'sectors_used': sectors_used,
  'depth_ppm': depth_ppm,
  'depth_err_ppm': depth_err_ppm,
  'status': r.status,
  'flags': list(r.flags),
  'metrics': r.metrics,
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_err_ppm": 11.153641839855947,
  "depth_ppm": 264.57118442002604,
  "flags": [
    "MODEL_PREFERS_NON_TRANSIT"
  ],
  "metrics": {
    "aic_eb_like": -485417.13288106344,
    "aic_transit_only": -485404.0707287799,
    "aic_transit_sinusoid": -485474.502500759,
    "artifact_prior_combined_risk": 0.2401026934306569,
    "artifact_prior_period_alias_risk": 0.4802053868613138,
    "artifact_prior_scattered_light_risk": 0.0,
    "artifact_prior_sector_quality_risk": 0.0,
    "artifact_risk": 0.8,
    "bic_eb_like": -485391.7044839281,
    "bic_transit_only": -485395.59459640144,
    "bic_transit_sinusoid": -485432.1218388668,
    "model_competition_label": "SINUSOID",
    "winner": "transit_sinusoid",
    "winner_margin_bic": 36.527242465352174
  },
  "sectors_used": [
    82,
    83
  ],
  "status": "ok"
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from tess_vetter.plotting import plot_model_comparison

step_id = '25_gated_82_83_v16_model_competition'
fname = 'V16_model_competition.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    plot_model_comparison(r, ax=ax)
    ax.set_title('V16 (gated 82+83): Model competition')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/25_gated_82_83_v16_model_competition/V16_model_competition.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/25_gated_82_83_v16_model_competition/V16_model_competition.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/25_gated_82_83_v16_model_competition/V16_model_competition.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** `MODEL_PREFERS_NON_TRANSIT`.
- **Result:** winner remains `transit_sinusoid` with a strong BIC margin (ΔBIC ≈ 36.5).
- **Why this is useful:** confirms the model-competition warning is not solely driven by the excluded sectors.
- **Interpretation:** treat as persistent evidence that variability/systematics are materially involved.
- **Next step:** V17 (gated) to re-check phase specificity and period neighborhood on the gated dataset.

</details>
